<a href="https://colab.research.google.com/github/pabloochocobar/CE888/blob/main/Tweet_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import classification_report
import argparse
import os

TASKS = [
    'emotion',
    'sentiment',
    'stance']

STANCE_TASKS = [
    'abortion',
    'atheism',
    'climate',
    'feminist',
    'hillary']


def load_gold_pred(args):
    tweet_classification_path = args.tweet_classification_path
    predictions_path = args.predictions_path
    task = args.task

    if 'stance' in task:
        gold = []
        pred = []
        for stance_t in STANCE_TASKS:
            gold_path = os.path.join(tweet_classification_path, task, stance_t, 'test_labels.txt')
            pred_path = os.path.join(predictions_path, task, stance_t + '.txt')
            gold.append(open(gold_path).read().split("\n")[:-1])
            pred.append(open(pred_path).read().split("\n")[:-1])
        # flatten lists of lists
        gold = [p for each_target in gold for p in each_target]
        pred = [p for each_target in pred for p in each_target]
    else:
        gold_path = os.path.join(tweet_classification_path, task, 'test_labels.txt')
        pred_path = os.path.join(predictions_path, task + '.txt')
        gold = open(gold_path).read().split("\n")[:-1]
        pred = open(pred_path).read().split("\n")[:-1]

    return gold, pred


def single_task_results(args):
    task = args.task
    tweet_classification_result = -1
    results = {}

    try:
        gold, pred = load_gold_pred(args)
        results = classification_report(gold, pred, output_dict=True)

            # Emotion (Macro f1)
        if 'emotion' in task:
            tweet_classification_result = results['macro avg']['f1-score']

            # Sentiment (Macro Recall)
        elif 'sentiment' in task:
            tweet_classification_result = results['macro avg']['recall']

        # Stance (Macro F1 of 'favor' and 'against' classes)
        elif 'stance' in task:
            f1_against = results['1']['f1-score']
            f1_favor = results['2']['f1-score']
            tweet_classification_result = (f1_against + f1_favor) / 2

    except Exception as ex:
        print(f"Issues with task {task}: {ex}")

    return tweet_classification_result, results


def is_all_good(all_tweet_classification_results):
    return all([r != -1 for r in all_tweet_classification_results.values()])


if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Tweet_classification script.')

    parser.add_argument('--tweet_classification_path', default="./datasets/", type=str, help='Path to Tweet_classification datasets')
    parser.add_argument('--predictions_path', default="./predictions/", type=str, help='Path to predictions files')
    parser.add_argument('--task', default="", type=str,
                        help='Indicate this parameter to get single task detailed results')

    args = parser.parse_args()

    if args.task == "":
        all_tweet_classification_results = {}

        # Results for each task
        for t in TASKS:
            args.task = t
            all_tweet_classification_results[t], _ = single_task_results(args)

        # Print results (score=-1 if some results are missing)
        print(f"{'-' * 30}")
        if is_all_good(all_tweet_classification_results):
            tweet_classification_final_score = sum(all_tweet_classification_results.values()) / len(all_tweet_classification_results.values())
        else:
            tweet_classification_final_score = -1
        for t in TASKS:
            # Each score
            print(f"{t}: {all_tweet_classification_results[t]}")
            # Final score
        print(f"{'-' * 30}\nTweet_classification Score: {tweet_classification_final_score}")

    else:
        # Detailed results of one single task (--task parameter)
        tweet_classification_resut, results = single_task_results(args)
        for k in results:
            print(k, results[k])
        print(f"{'-' * 30}\nTweet_classification Score ({args.task}): {tweet_classification_resut}")

usage: ipykernel_launcher.py [-h] [--tweeteval_path TWEETEVAL_PATH]
                             [--predictions_path PREDICTIONS_PATH]
                             [--task TASK]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-fa9fa764-4728-488b-b0c3-589c4e85006e.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
